The purpose of this notebook is to make a **very** basic transformer model that predicts load values. For this initial model, I intend to **only** use previous readings as input. That is, **no context variables (weather, time, etc)** will be used here.

In [16]:
import torch
import torch.nn as nn
import torch.utils.data
from torch.nn import functional as F
import pandas as pd
import numpy as np
import random

In [17]:
# -------
EPOCHS = 10
LR = 0.001
SEQ_LENGTH = 12 # Number of historical data points to consider
BATCH_SIZE = 64
D_MODEL = 1 #2  # number of features (demand + temperature)
NHEAD = 1
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2
# -------

In [18]:
def create_sequences(data, seq_length, num_samples):
    sequences = []
    target = []
    if (num_samples > len(data)):
        print("num_samples too large")
        return
    
    for _ in range(num_samples):
        idx = random.randint(0, len(data)-seq_length - 1)
        seq = data[idx:idx+seq_length]
        label = data[idx+seq_length]
        sequences.append(seq)
        target.append(label)

    return np.array(sequences), np.array(target)

In [97]:
class TransformerPredictor(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dropout=0.1):
        super(TransformerPredictor, self).__init__()
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, dropout=dropout),
            num_layers=num_encoder_layers
        )
        self.fc = nn.Linear(d_model, 1)

    def forward(self, src):
        src = src.transpose(0, 1)  # This transposes the batch and sequence dimensions.
#         print(src.shape)
        output = self.transformer_encoder(src)  
#         print(output.shape)
        last_output = output[-1, :, :]
#         print(last_output.shape)
        prediction = self.fc(last_output)
#         print(prediction.shape)
#         print(prediction.squeeze(-1).shape)
        return prediction.squeeze(-1)
          

In [98]:
# Load your data into a DataFrame
path = "/Users/aidanwiteck/Desktop/Princeton/Year 4/Thesis/electricgrid/data/final_tables/banc/banc.csv"
df = pd.read_csv(path)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Z-score normalization
mean_demand = df['Demand (MWh)'].mean()
std_demand = df['Demand (MWh)'].std()

df['Normalized Demand'] = (df['Demand (MWh)'] - mean_demand) / std_demand

In [99]:
# Create sequences
X, y = create_sequences(df['Normalized Demand'].values, SEQ_LENGTH, 5000)  # For example, creating 5000 samples
X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

In [100]:
# Split data (80/20 split)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [101]:
X_train.shape, y_train.shape, 

(torch.Size([4000, 12, 1]), torch.Size([4000, 1]))

In [102]:
# X_train

In [103]:
# Data loaders
train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)

In [104]:
# Model, Loss, Optimizer
model = TransformerPredictor(D_MODEL, NHEAD, NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

/Users/aidanwiteck/.virtualenvs/thesis/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [105]:
# Training loop
model.train()
for epoch in range(EPOCHS):
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        outputs = model(inputs).unsqueeze(-1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{EPOCHS} | Loss: {loss.item()}")

Epoch 1/10 | Loss: 0.7171139717102051
Epoch 2/10 | Loss: 0.35124412178993225
Epoch 3/10 | Loss: 0.15351220965385437
Epoch 4/10 | Loss: 0.4189613163471222
Epoch 5/10 | Loss: 0.36694180965423584
Epoch 6/10 | Loss: 0.3504642844200134
Epoch 7/10 | Loss: 0.14471067488193512
Epoch 8/10 | Loss: 24.719085693359375
Epoch 9/10 | Loss: 0.404633104801178
Epoch 10/10 | Loss: 0.3308394253253937


# Evaluate

In [106]:
def generate_load(model, input_sequence, num_predictions):
    """
    Generate load values using the trained model.
    
    Parameters:
    - model: The trained model
    - input_sequence: A starting sequence to use for generation
    - num_predictions: Number of future load values to predict

    Returns:
    A list of predicted load values.
    """
    model.eval()  # Set the model to evaluation mode
    predictions = []

    for _ in range(num_predictions):
        with torch.no_grad():
            prediction = model(input_sequence)
            print(f"Prediction shape: {prediction.shape}")  # Add this line
            predictions.append(prediction.item())

            # Update the input_sequence for the next prediction
            input_sequence = torch.roll(input_sequence, shifts=-1, dims=0)
            input_sequence[-1, 0] = prediction

    return predictions

In [111]:
# Assuming you've trained the model

# Let's predict the next 10 values using the last sequence from X_test as a starting point
starting_sequence = X_test[3].unsqueeze(0)  # unsqueeze to add batch dimension
print(starting_sequence)
num_predictions = 10

predicted_load_values = generate_load(model, starting_sequence, num_predictions)
print(predicted_load_values)

tensor([[[-0.5240],
         [-0.3843],
         [-0.2858],
         [-0.2673],
         [-0.2998],
         [-0.3226],
         [-0.3670],
         [-0.3594],
         [-0.3735],
         [-0.3594],
         [-0.3464],
         [-0.2793]]])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
Prediction shape: torch.Size([1])
[0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636, 0.00014697760343551636]


In [50]:
def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    predictions, true_values = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch[0])
            print(X_batch.shape)
            print(outputs.shape)
            print(y_batch.shape)
            loss = criterion(outputs, y_batch)
            
            test_loss += loss.item()

            predictions.extend(outputs.numpy())
            true_values.extend(y_batch.numpy())

    # Calculate the average loss over the entire test dataset
    average_test_loss = test_loss / len(test_loader)
    
    return average_test_loss, predictions, true_values

# Create a DataLoader for the test dataset
test_data = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)

# Evaluate the model
test_loss, predictions, true_values = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}")

# If you want other metrics, you can compute them using `predictions` and `true_values`.
# For example, you could calculate the MAE, RMSE, etc.


IndexError: too many indices for tensor of dimension 2

In [9]:
df

,timestamp,Demand (MWh),Demand Forecast (MWh),Net Generation (MWh),Region,temperature,humidity,cloudcover,windspeed
0,2015-07-01 01:00:00,2513.0,2226.0,1559.0,banc,38.5,18,10,8.7
1,2015-07-01 02:00:00,2275.0,2035.0,1441.0,banc,37.7,19,12,10.4
2,2015-07-01 03:00:00,2104.0,1897.0,1399.0,banc,35.4,23,11,10.7
3,2015-07-01 04:00:00,1988.0,1821.0,1354.0,banc,32.5,27,16,11.4
4,2015-07-01 05:00:00,1958.0,1811.0,1334.0,banc,30.3,31,21,9.0
...,...,...,...,...,...,...,...,...,...
65779,2022-12-31 20:00:00,2025.0,2021.0,1539.0,banc,12.5,92,100,11.7
65780,2022-12-31 21:00:00,1821.0,1941.0,1503.0,banc,12.3,91,100,14.0
65781,2022-12-31 22:00:00,1686.0,1862.0,1488.0,banc,12.1,91,100,17.2
65782,2022-12-31 23:00:00,1625.0,1784.0,1570.0,banc,12.1,91,100,19.6
